<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla, Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias, Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Carolina Salgado
- Nombre de alumno 2: Sebastián Urbina


### **Link de repositorio de GitHub:** `https://github.com/SebasUrbina/MDS7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
# try:
#     from google.colab import drive
#     drive.mount("/content/drive")
#     path = 'Dirección donde tiene los archivos en el Drive'
# except: 
#     print('Ignorando conexión drive-colab')

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [3]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [4]:
def custom_features(dataframe_in):

    # Copia del dataframe original
    data = dataframe_in.copy()

    # Fecha de hoy
    today = datetime.datetime.today()

    # Agregamos una columna del ingreso por venta del producto
    data["Income"] = data["Quantity"]*data["Price"]

    data = data.astype(
        {
            "Invoice": "object",
            "Customer ID": "object",
        }
    )

    data["InvoiceDate"] = data["InvoiceDate"].dt.date
    data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])

    length = data.groupby("Customer ID")["InvoiceDate"].agg(np.ptp).reset_index().rename(columns={"InvoiceDate": "Length"})
    length["Length"] = length["Length"].dt.days

    recency = data.groupby("Customer ID")["InvoiceDate"].agg(lambda x: today - np.max(x)).reset_index().rename(columns={"InvoiceDate": "Recency"})
    recency["Recency"] = recency["Recency"].dt.days

    income_per_invoice = data.groupby(["Customer ID", "Invoice"])['Income'].sum().reset_index()
    monetary = income_per_invoice.groupby("Customer ID")["Income"].mean().reset_index().rename(columns={"Income": "Monetary"})

    frequency = data.groupby("Customer ID")["Invoice"].agg("count").reset_index().rename(columns={"Invoice": "Frequency"})

    invoice_date = data.groupby(["Customer ID", "Invoice"])["InvoiceDate"].agg("min").reset_index()
    invoice_date["InvoiceDate_diff"] = invoice_date.groupby("Customer ID")["InvoiceDate"].agg("diff")
    invoice_date = invoice_date.dropna()
    invoice_date["InvoiceDate_diff"] = invoice_date["InvoiceDate_diff"].dt.days
    periodicity = invoice_date.groupby("Customer ID")["InvoiceDate_diff"].std(ddof=0).reset_index().rename(columns={"InvoiceDate_diff": "Periodicity"})
    
    result = length.merge(recency,on='Customer ID').merge(monetary,on='Customer ID').merge(frequency,on='Customer ID').merge(periodicity,on='Customer ID')

    return result
    

In [5]:
df_LRMPF = custom_features(df_retail)
df_LRMPF.head()

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity
0,12346.0,196,4721,33.896364,33,34.745359
1,12347.0,37,4559,661.660000,71,0.000000
2,12349.0,182,4599,890.380000,102,72.000000
3,12352.0,17,4567,171.900000,18,0.000000
4,12356.0,44,4572,1186.766667,83,9.000000


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [6]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        return self.scaler.transform(X)


Aplicamos la transformación al set de datos de LRMFP.

In [7]:
# Definir las columnas a transformar (todas menos Customer ID)
columns_to_transform = df_LRMPF.columns[1:]

# Crear el ColumnTransformer con MinMaxScaler
preprocessor = ColumnTransformer(
    transformers=[('minmax', MinMax(), columns_to_transform)])

# Aplicar la transformación a las columnas seleccionadas
df_LRMPF_p = preprocessor.fit_transform(df_LRMPF)

# Ver los resultados
df_LRMPF_p = pd.DataFrame(df_LRMPF_p, columns=columns_to_transform)

In [8]:
df_LRMPF_p.head()

,Length,Recency,Monetary,Frequency,Periodicity
0,0.525469,0.439678,0.003086,0.005570,0.191963
1,0.099196,0.005362,0.087033,0.012397,0.000000
2,0.487936,0.112601,0.117618,0.017966,0.397790
3,0.045576,0.026810,0.021540,0.002875,0.000000
4,0.117962,0.040214,0.157252,0.014553,0.049724


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [9]:
tsne = TSNE(n_components=2, random_state=42)

pipeline_LRMPF = Pipeline([
    ("LRMPF", FunctionTransformer(custom_features)),
    ("scaler", preprocessor),
    ("TSNE", tsne)
])

In [10]:
LRMPF_tsne = pipeline_LRMPF.fit_transform(df_retail)
LRMPF_tsne

array([[ 15.56617  ,  28.282047 ],
       [ 56.77573  , -22.076345 ],
       [ -1.9166182,  27.5221   ],
       ...,
       [-22.092728 ,  11.425085 ],
       [  7.620004 ,  -1.3391734],
       [ -3.8368108,  18.534449 ]], dtype=float32)

In [11]:
x_tsne = LRMPF_tsne[:, 0]
y_tsne = LRMPF_tsne[:, 1]

px.scatter(x=x_tsne, y=y_tsne)

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [12]:
def calculate_inertia(k):
    pipeline = Pipeline([
        ("LRMPF", FunctionTransformer(custom_features)),
        ("scaler", preprocessor),
        ('kmeans', KMeans(n_clusters=k, random_state=42))
    ])
    cluster_labels = pipeline.fit_predict(df_retail)
    inertia = pipeline.named_steps['kmeans'].inertia_
    return inertia

# Crear una lista de valores de k en el rango [1, 20]
k_values = list(range(1, 21))

intertias = [
    [i, calculate_inertia(i)]
    for i in k_values
]

c:\Users\carol\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\carol\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\carol\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\carol\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\carol\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_i

In [13]:
intertias = pd.DataFrame(intertias, columns=["k clusters", "inertia"])
intertias.head(10)

,k clusters,inertia
0,1,462.431997
1,2,220.975995
2,3,167.119443
3,4,128.688274
4,5,102.724472
5,6,91.349768
6,7,81.748216
7,8,73.645039
8,9,68.669097
9,10,63.870577


In [14]:
px.line(
    intertias,
    x="k clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
)

De acuerdo a la visualización y haciendo uso de la lógica del método del codo, cinco clusters parecen adecuados para ejecutar el algoritmo de K-means.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [15]:
pipeline_kmeans = Pipeline([
        ("LRMPF", FunctionTransformer(custom_features)),
        ("scaler", preprocessor),
        ('kmeans', KMeans(n_clusters=5, random_state=42))
    ])

df_clients = pipeline_kmeans.named_steps['LRMPF'].transform(df_retail)
cluster_labels = pipeline_kmeans.fit_predict(df_retail)

df_clients["cluster"] = cluster_labels

c:\Users\carol\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [16]:
df_clients

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity,cluster
0,12346.0,196,4721,33.896364,33,34.745359,4
1,12347.0,37,4559,661.660000,71,0.000000,1
2,12349.0,182,4599,890.380000,102,72.000000,4
3,12352.0,17,4567,171.900000,18,0.000000,1
4,12356.0,44,4572,1186.766667,83,9.000000,1
...,...,...,...,...,...,...,...
2888,18276.0,337,4584,264.132000,80,32.690786,0
2889,18277.0,97,4589,267.417500,56,33.209771,1
2890,18283.0,276,4574,103.228333,217,55.358468,2
2891,18286.0,247,4668,648.215000,67,0.000000,4


In [17]:
df_clients.groupby("cluster")["Length", "Recency", "Monetary", "Frequency", "Periodicity"].mean().round(2)

C:\Users\carol\AppData\Local\Temp\ipykernel_36172\4078115753.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Length,Recency,Monetary,Frequency,Periodicity
cluster,,,,,
0,328.58,4577.01,434.17,251.68,25.42
1,56.55,4596.84,383.54,66.47,5.46
2,285.49,4592.79,346.84,93.65,86.50
3,78.52,4774.78,378.92,49.38,8.93
4,197.56,4615.37,370.77,97.41,23.95


A continuación, se presentan las características principales de cada grupo de clientes determinado por la clusterización:

* Grupo 0: Tiene el mayor valor en $Length$ y $Frequency$, por lo tanto, se podría interpretar que son los clientes más fieles. En $Recency$ tiene un valor similar a los grupos 1, 2 y 4. Además, tiene el mayor valor de $Monetary$, lo que significa que es el segmento que más aporta monetariamente a la empresa. Sin embargo, tiene una $Periodicity$ de $25.42$, que significa una visita regular pero no tanto así como el segmento 2.

    En resumen, este segmento podría identificarse como aquellos clientes que visitan con alta frecuencia la tienda y gastan bastante en sus compras.

* Grupo 1: Tiene el menor valor de $Length$, por lo tanto, no es un cliente fiel en el sentido de que vuelva a la tienda tras un largo tiempo. En $Recency$ tiene un valor similar a los grupos 0, 2 y 4. Además, tiene el segundo mayor valor de $Monetary$, lo que significa que es el segundo segmento que más aporta monetariamente a la empresa. Por otro lado, no tiene una $Frequency$ particularmente alta en comparación al resto de los grupos, lo que quiere decir que no es un cliente muy frecuente. Y, de hecho, es el grupo con la menor $Periodicity$.

    En este caso, el grupo podría reconocerse como los clientes que tienen una visita más ocasional y poco frecuente a la tienda, pero que realiza compras que realiza compras un poco más costosas que la mayoría de los segmentos, salvo el grupo 0.

* Grupo 2: Tiene el segundo mayor valor en $Length$, por lo tanto, es un cliente fiel en el sentido de que vuelve a la tienda tras un largo tiempo. En $Recency$ tiene un valor similar a los grupos 0, 1 y 4. Tiene el menor valor de $Monetary$, por lo tanto, no es un cliente que contribuya demasiado en términos de ingresos a la empresa. No tiene una $Frequency$ alta o baja en relación al resto de los grupos. Pero tiene la mayor de las $Periodicity$, o sea, tiene una alta regularidad de visita a la tienda.

    Aquí se podría identificar al grupo de clientes que es regular en la tienda, aunque no se caracteriza por su frecuencia y tampoco gasta mucho en sus compras.

* Grupo 3: Tiene una $Length$ relativamente baja en comparación al valor del resto de los grupos. Tiene el mayor valor de $Recency$, o sea, es el tipo de cliente que hizo su última compra hace mucho tiempo atrás. Tiene un valor de $Monetary$ que no es particularmente alto ni bajo en términos relativos. Por otro lado, tiene la menor $Frequency$, lo que significa que es el grupo que menos ha visitado la tienda. Y, en ese sentido, tiene un bajo valor de $Periodicity$, así que no es un cliente regular de la empresa.

    Este tipo de clientes puede reconocerse como un segmento que no es fiel a la empresa en el sentido de que su frecuencia de compra en la tienda es bastante baja. Por lo tanto, pareciera que sus registros corresponden a compras ocasionales.

* Grupo 4: Tiene una $Length$ ni alta ni baja en relación a los demás grupos. En $Recency$ tiene un valor similar a los grupos 0, 1 y 2. Tiene un valor de $Monetary$ que no es particularmente alto ni bajo en términos relativos. No tiene una $Frequency$ alta o baja en relación al resto de los grupos. Y tiene una $Periodicity$ tampoco alta ni baja.

    En general, podría decirse que este es un cliente bastante promedio, que no se destaca por algún atributo particular.

Dada la caracterización anterior, se puede concluir que efectivamente el clustering arroja agrupaciones coherentes, ya que se puede identificar a tipos de clientes que hacen sentido con la realidad.  

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [18]:
df_clients["x_tsne"] = x_tsne
df_clients["y_tsne"] = y_tsne

In [19]:
px.scatter(df_clients, x="x_tsne", y="y_tsne", color="cluster")

Esta visualización muestra una clara segmentación de los datos dada por el algoritmo de clustering K-means. Lo cual guarda sentido con la buena calidad de las distinciones a nivel de caracterización de los segmentos, como se hizo en la Parte 1.3.3.2.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>